# Data Import

In [ ]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timezone

# make sure root/ is on the import path
# (assuming this notebook lives in root/preprocessing/model_optimization/)
BASE_DIR = Path().resolve().parents[1]
sys.path.append(str(BASE_DIR))

DATA_DIR = BASE_DIR / "data" / "final"  # e.g. root/data/diets.csv, root/data/exercises.csv

# replace with the actual filenames
_diets_df = pd.read_csv(DATA_DIR / "processed_final_diets.csv")
_ex_df    = pd.read_csv(DATA_DIR / "processed_final_exercises.csv")

print(f"Loaded {len(_diets_df)} diets and {len(_ex_df)} exercises")


## Filtering functions

In [ ]:
def filter_diets(user_params: dict) -> pd.DataFrame:
    df = _diets_df.copy()
    dr = [d.lower() for d in user_params.get("dietRestrictions", [])]
    if "none" not in dr:
        df = df[df["diet_type"].str.lower().isin(dr)]
    vp = [v.lower() for v in user_params.get("varietyPreferences", [])]
    if "none" not in vp:
        df = df[df["cuisine_type"].str.lower().isin(vp)]
    print(f"Filtered diets ({len(df)} rows)")
    return df

def filter_exercises(user_params: dict) -> pd.DataFrame:
    df = _ex_df.copy()
    fl = user_params.get("fitnessLevel", "").lower()
    if fl and fl != "none":
        df = df[df["difficulty_level"].str.lower() == fl]
    pl = user_params.get("preferredLocation", "").lower()
    if pl and pl != "none":
        df = df[df["workout_location"].str.lower() == pl]
    wt = user_params.get("preferredWorkoutType", "").lower()
    if wt and wt != "none":
        df = df[df["activity_type"].str.lower() == wt]
    print(f"Filtered exercises ({len(df)} rows)")
    return df

## Pre-Calculations functions

In [ ]:
def compute_user_metrics(user: dict) -> dict:
    w, h, a = user["weight"], user["height"], user["age"]
    s = 5 if user["gender"].lower() == "male" else -161

    # 1. BMR
    bmr = 10 * w + 6.25 * h - 5 * a + s

    # 2. Activity multipliers (all lower-case keys)
    activity_map = {
        "Sedentary": 1.2,
        "Lightly Active": 1.375,
        "Moderately Active": 1.55,
        "Very Active": 1.725,
        "Extra Active": 1.9,
    }
    act = user["activityLevel"].lower()
    multiplier = activity_map.get(act, 1.2)
    tdee = bmr * multiplier

    # 3. Weight change & daily calorie delta
    gw = user["goalWeight"]
    wt_change = gw - w
    today = datetime.now(timezone.utc)
    tgt   = datetime.fromisoformat(user["goalTargetDate"].replace("Z","+00:00"))
    days  = max((tgt - today).days, 1)
    cal_delta = 7700 * wt_change / days

    # 4. Target calories per day
    target_cal_pd = tdee + cal_delta

    return {
        "BMR": round(bmr,2),
        "TDEE": round(tdee,2),
        "weight_change_kg": round(wt_change,2),
        "days_to_target": days,
        "calorie_change_per_day": round(cal_delta,2),
        "target_calorie_per_day": round(target_cal_pd,2),
    }

## Defining sample user (example request body from frontend)

In [ ]:
sample_user = {
    "activityLevel": "Sedentary",
    "age": 41,
    "allergies": [],
    "daysWeek": 4,
    "dietRestrictions": ["none"],
    "fitnessLevel": "beginner",
    "freeTime": 4,
    "gender": "male",
    "goalTargetDate": "2025-04-22T04:48:00.000Z",
    "goalType": "weight_gain",
    "goalWeight": 65,
    "height": 170,
    "mealPrepTime": 15,
    "mealsPerDay": 5,
    "name": "wei",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 70
}

In [ ]:
# filter data
diets_df = filter_diets(sample_user)
ex_df    = filter_exercises(sample_user)

# compute metrics
metrics = compute_user_metrics(sample_user)

# convert to lists of dicts for easy iteration in Gurobi
diet_options = diets_df.to_dict(orient="records")
ex_options   = ex_df.to_dict(orient="records")

print("User metrics:", metrics)
print(f"{len(diet_options)} diet options, {len(ex_options)} exercise options ready for modeling.")

## Gurobi model formulation